In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import os
import pickle
import pandas as pd
import financial.data as fd
import financial.momentum.storeLocalModel as sLM
from financial.momentum.models.exponentialRegression import ExponentialRegressionModelFactory
from financial.io.file.cache import FileCache

from dotenv import load_dotenv
from financial.momentum.utilities import find_dotenv
load_dotenv(dotenv_path=find_dotenv())

ticker = "^GSPC"
cache_path=os.environ["CACHE"]+"/"  
model_name = "exponential"

ds = fd.CachedDataStore(path=os.environ.get("DATA", "./data"), cache=FileCache(cache_path=cache_path))

def local_features_wrapper(ds: fd.DataStore) -> fd.Set:
    '''
    Wrapper function to generate local regression features.
    This is required by ModelFactory when creating a model.
    '''
    return sLM.local_features(ds, ticker)


In [4]:
# Ejecutar la función para almacenar los datos
print(f"⚙️  Almacenando datos de momentum para {ticker}...")
sLM.store_exponentialModel_data(ticker)

⚙️  Almacenando datos de momentum para ^GSPC...


In [10]:
# Intentar recuperar los archivos guardados
def print_exponentialModel_data(ticker, ds, cathe_path):
    '''
    prediction_path = slope_path = os.path.join(cache_path, f"model/momentum/{model_name}/{ticker}.pkl")
    slope_path = os.path.join(cache_path, f"model/momentum/{model_name}/{ticker}@slope.pkl")
    r2_path = os.path.join(cache_path, f"model/momentum/{model_name}/{ticker}@r2.pkl")
    '''

    prediction_path = os.path.join(cache_path, f"model-momentum-{model_name}-{ticker}")
    slope_path = os.path.join(cache_path, f"model-momentum-{model_name}-{ticker}@slope")
    r2_path = os.path.join(cache_path, f"model-momentum-{model_name}-{ticker}@r2")
    
    # Comprobar si los archivos existen
    if os.path.exists(prediction_path) and os.path.exists(slope_path) and os.path.exists(r2_path):
    
        start_date = '1990-01-20'
        end_date = '2020-12-31'
        data = ds.get_data(ticker, start_date, end_date)
    
        #print("\n📊 **Data Series")
        #print(data)

        series = pd.read_pickle(prediction_path)
        print(type(series))
        print(series.head())
        print(series.index)
        print(series.name)

        series = pd.read_pickle(slope_path)
        print(type(series))
        print(series.head())
        print(series.index)
        print(series.name)

        series = pd.read_pickle(r2_path)
        print(type(series))
        print(series.head())
        print(series.index)
        print(series.name)
        
        
        # Cargar los archivos y comprobar los datos
        with open(prediction_path, 'rb') as file:
            prediction_series = pickle.load(file)
            print(type(prediction_series))
            print("\n📊 **Prediction Series")
            print(prediction_series.apply(lambda x: f"{x:.15f}") + "%")
            print(prediction_series)
        
        with open(slope_path, 'rb') as file:
            slope_series = pickle.load(file)
            print(type(slope_series))
            print("\n📊 **Slope Series")
            print(slope_series.apply(lambda x: f"{x:.15f}"))
    
    
        with open(r2_path, 'rb') as file:
            r2_series = pickle.load(file)
            print(type(r2_series))
            print("\n📊 **R² Series")
            print(r2_series)

        return r2_series

    else:
        print("❌ ERROR: No se encontraron los archivos de caché.")



In [11]:
r2_series = print_exponentialModel_data("bellota", ds, cache_path)

❌ ERROR: No se encontraron los archivos de caché.


In [ ]:
import matplotlib.pyplot as plt

# Seleccionar un periodo más corto (por ejemplo, 2020-2021)
start_date = "2019-01-01"
end_date = "2020-12-31"

fig, ax = plt.subplots(2, 1, figsize=(16, 9), sharex=True)
data = ds.get_data(ticker, start_date, end_date)
# Primer gráfico: Evolución de los precios
ax[0].plot(data, label="Precio", color="blue")
ax[0].set_title("Evolución del precio")
ax[0].legend()

# Segundo gráfico: R² en el mismo periodo
ax[1].plot(r2_series[start_date:end_date], label="R²", color="red")
ax[1].set_title("Evolución de R²")
ax[1].legend()

plt.xlabel("Fecha")
plt.show()

In [ ]:
ticker = "^GSPC"
print(f"⚙️  Almacenando datos de momentum para {ticker}...")
sLM.store_exponentialModel_data(ticker, ds, cache_path)


In [ ]:
print_exponentialModel_data(ticker, ds, cache_path)

In [13]:
universe_ing_eur = ['ZPDM', 'ZPDI', 'ZPDS', 'ZPDH', 'ZPDT', 'ZPDD', 'ZPDF',
                   'SPY5', 'DJE', 'UST', 'LYTR', 'IQQH']

universe_ferrer = ['XLI', 'XLF', 'XLV', 'XLK', 'XLC', 'XLU', 'XLY', 'XLB', 'XLE', 'XLP',
                    'TLT', 'TIP', 'IEF', 'SHY', 'AGG', 'VNQ', 'GLD', 'DBC']

for ticker in universe_ferrer:
    prediction_path = os.path.join(cache_path, f"model-momentum-exponential-{ticker}")
    if os.path.exists(prediction_path):
        continue
    else:
        sLM.store_exponentialModel_data(ticker, ds, cache_path)
    #print_exponentialModel_data(ticker, ds, cache_path)